# Nearest Neighbors

Nearest Neighbors enables the query of the k-nearest neighbors from a set of input samples.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input. 

For information on converting your dataset to cuDF format, refer to the cuDF documentation: https://docs.rapids.ai/api/cudf/stable

For additional information on cuML's Nearest Neighbors implementation: https://docs.rapids.ai/api/cuml/stable/api.html#nearest-neighbors

In [1]:
import cudf
import numpy as np
from cuml.datasets import make_blobs
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
from sklearn.neighbors import NearestNeighbors as skNearestNeighbors

## Define Parameters

In [2]:
n_samples = 2**17
n_features = 40

n_query = 2**13
n_neighbors = 4
random_state = 0

In [3]:
n_samples, n_query

(131072, 8192)

## Generate Data

### GPU

In [4]:
%%time
device_data, _ = make_blobs(n_samples=n_samples,
                            n_features=n_features,
                            centers=5,
                            random_state=random_state)

device_data = cudf.DataFrame(device_data)

CPU times: user 11.7 s, sys: 36.1 s, total: 47.8 s
Wall time: 48.3 s


In [5]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
host_data = device_data.to_pandas()

## Scikit-learn Model

## Fit

In [6]:
%%time
knn_sk = skNearestNeighbors(algorithm="brute",
                            n_jobs=-1)
knn_sk.fit(host_data)

CPU times: user 41 ms, sys: 162 ms, total: 203 ms
Wall time: 201 ms


NearestNeighbors(algorithm='brute', n_jobs=-1)

In [7]:
%%time
D_sk, I_sk = knn_sk.kneighbors(host_data, n_neighbors)

CPU times: user 8min 38s, sys: 19.1 s, total: 8min 57s
Wall time: 36.7 s


## cuML Model

### Fit

In [8]:
%%time
knn_cuml = cuNearestNeighbors()
knn_cuml.fit(device_data)

CPU times: user 118 ms, sys: 0 ns, total: 118 ms
Wall time: 121 ms


NearestNeighbors()

In [9]:
%%time
D_cuml, I_cuml = knn_cuml.kneighbors(device_data, n_neighbors)

CPU times: user 33.2 s, sys: 79.8 ms, total: 33.3 s
Wall time: 32.8 s


## Compare Results

cuML currently uses FAISS for exact nearest neighbors search, which limits inputs to single-precision. This results in possible round-off errors when floats of different magnitude are added. As a result, it's very likely that the cuML results will not match Sciklearn's nearest neighbors exactly. You can read more in the [FAISS wiki](https://github.com/facebookresearch/faiss/wiki/FAQ#why-do-i-get-weird-results-with-brute-force-search-on-vectors-with-large-components).

### Distances

In [9]:
passed = np.allclose(D_sk, D_cuml.to_numpy(), atol=1e-3)
print('compare knn: cuml vs sklearn distances %s'%('equal'if passed else 'NOT equal'))

compare knn: cuml vs sklearn distances NOT equal


### Indices

In [10]:
sk_sorted = np.sort(I_sk, axis=1)
cuml_sorted = np.sort(I_cuml.to_numpy(), axis=1)

diff = sk_sorted - cuml_sorted

passed = (len(diff[diff!=0]) / n_samples) < 1e-9
print('compare knn: cuml vs sklearn indexes %s'%('equal'if passed else 'NOT equal'))

compare knn: cuml vs sklearn indexes NOT equal
